In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
import os
from habnet.featuriser.featurise import Featuriser, MOL_TYPES
from habnet.featuriser.habnet_featurizer import AtomHAbNetFeaturizer
from chemprop import data, featurizers, nn
from chemprop.nn import metrics
from chemprop.models import multi
from chemprop.nn.metrics import MSE
from lightning import pytorch as pl
from chemprop.featurizers import MorganBinaryFeaturizer
# Logging setup
from lightning import pytorch as pl
from sklearn.preprocessing import PowerTransformer
pl.seed_everything(42)  # For reproducibility

Seed set to 42


42

In [41]:
sdf_path = '/home/calvin/code/chemprop_phd_customised/habnet/data/processed/sdf_data'
target_data = '/home/calvin/code/chemprop_phd_customised/habnet/data/processed/target_data/kinetics_summary.csv'
assert os.path.exists(sdf_path), f"SDF path does not exist: {sdf_path}"
assert os.path.exists(target_data), f"Target data file does not exist: {target_data}"

In [42]:
import pandas as pd
pd.read_csv(target_data)

,rxn,label,A,A_units,n,Ea,Ea_units,T0,Tmin,Tmax,dA,dn,dEa,source_comment
0,kfir_rxn_13712,k_rev (TST),1.482350e+01,cm^3/(mol*s),3.49251,88.52070,kJ/mol,1.0 K,300.0 K,3000.0 K,1.03737,0.004779,0.027328,"Fitted to 50 data points; dA = *|/ 1.03737, dn..."
1,kfir_rxn_13712,k_rev (TST+T),5.604340e-02,cm^3/(mol*s),4.16021,79.03880,kJ/mol,1.0 K,300.0 K,3000.0 K,1.34250,0.038364,0.219391,"Fitted to 50 data points; dA = *|/ 1.3425, dn ..."
2,kfir_rxn_13712,k_for (TST+T),3.501510e-02,cm^3/(mol*s),4.19628,4.50494,kJ/mol,1.0 K,300.0 K,3000.0 K,1.33959,0.038081,0.217772,"Fitted to 50 data points; dA = *|/ 1.33959, dn..."
3,kfir_rxn_10214,k_rev (TST),1.209950e+00,cm^3/(mol*s),3.43877,36.09200,kJ/mol,1.0 K,300.0 K,3000.0 K,1.16517,0.019911,0.113868,"Fitted to 50 data points; dA = *|/ 1.16517, dn..."
4,kfir_rxn_10214,k_rev (TST+T),1.436700e-04,cm^3/(mol*s),4.53308,22.87440,kJ/mol,1.0 K,300.0 K,3000.0 K,2.14336,0.099300,0.567869,"Fitted to 50 data points; dA = *|/ 2.14336, dn..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5551,rxn_31,k_rev (TST+T),6.050880e-09,cm^3/(mol*s),5.14186,10.95990,kJ/mol,1.0 K,300.0 K,3000.0 K,2.40935,0.114537,0.655006,"Fitted to 50 data points; dA = *|/ 2.40935, dn..."
5552,rxn_31,k_for (TST+T),7.799430e-08,cm^3/(mol*s),5.46233,54.74430,kJ/mol,1.0 K,300.0 K,3000.0 K,2.73081,0.130850,0.748296,"Fitted to 50 data points; dA = *|/ 2.73081, dn..."
5553,rxn_1887,k_rev (TST),4.603860e-01,cm^3/(mol*s),3.40092,35.09730,kJ/mol,1.0 K,300.0 K,3000.0 K,1.04909,0.006242,0.035698,"Fitted to 50 data points; dA = *|/ 1.04909, dn..."
5554,rxn_1887,k_rev (TST+T),9.164750e-06,cm^3/(mol*s),4.70965,18.81960,kJ/mol,1.0 K,300.0 K,3000.0 K,1.97046,0.088345,0.505222,"Fitted to 50 data points; dA = *|/ 1.97046, dn..."


In [43]:
target_data_pd = pd.read_csv(target_data)
target_data_pd[target_data_pd['label'] == 'k_for (TST+T)'].describe()

,A,n,Ea,dA,dn,dEa
count,1.852000e+03,1852.000000,1852.000000,1852.000000,1852.000000,1852.000000
mean,1.219883e+09,4.328684,29.182837,1.903003,0.069866,0.399542
std,5.245628e+10,0.807664,31.318774,2.266644,0.050453,0.288526
min,4.856850e-30,1.563490,-7.952310,1.013580,0.001757,0.010050
25%,8.707215e-06,3.890185,6.807783,1.234810,0.027472,0.157105
50%,8.341260e-04,4.407760,18.278100,1.640895,0.064506,0.368890
75%,9.697212e-02,4.832607,43.309175,2.157915,0.100182,0.572911
max,2.257450e+12,11.588700,210.310000,85.778800,0.579848,3.315990


In [44]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PowerTransformer

# 0) Paths
raw_csv   = '/home/calvin/code/chemprop_phd_customised/habnet/data/processed/target_data/kinetics_summary.csv'
out_csv   = '/home/calvin/code/chemprop_phd_customised/habnet/data/processed/target_data/temp_target_kinetic_data.csv'

# 1) Load
df = pd.read_csv(raw_csv)

#  1.a) Filter for label == 'k_for (TST+T)'
df = df[df['label'] == 'k_for (TST+T)'].copy()

# 2) Drop any rows missing the columns we care about
df = df.dropna(subset=['rxn', 'label', 'A', 'n', 'Ea'])

# 3) Dedupe by (rxn, label), keeping the first occurrence
df = df.drop_duplicates(subset=['rxn','label'], keep='first')


# 4) Filter out non-positive A (cannot log-transform those)
df = df[df['A'] > 0].copy()


# 5) Log10-transform A into a new column
df['A_log10'] = np.log10(df['A'])

# 6) Fit & apply Yeo–Johnson transform to the entire Ea column
# Fit on a pure numpy array
Ea_vals = df['Ea'].to_numpy().reshape(-1, 1)   # ndarray, no column names
pt_ea = PowerTransformer(method='yeo-johnson')
Ea_yj = pt_ea.fit_transform(Ea_vals).ravel()
df['Ea_yj'] = Ea_yj

# 7) (Optional) inspect your transforms
print("A_log10 summary:\n", df['A_log10'].describe())
print("Ea_yj   summary:\n", df['Ea_yj'].describe())

# 8) Save out
df.to_csv(out_csv, index=False)
print(f"Wrote processed targets to {out_csv}")


A_log10 summary:
 count    1848.000000
mean       -2.736038
std         3.319707
min       -29.313645
25%        -5.060128
50%        -3.078768
75%        -1.013356
max        12.353618
Name: A_log10, dtype: float64
Ea_yj   summary:
 count    1848.000000
mean        0.000000
std         1.000271
min        -3.286445
25%        -0.618092
50%        -0.109654
75%         0.646328
max         3.498660
Name: Ea_yj, dtype: float64
Wrote processed targets to /home/calvin/code/chemprop_phd_customised/habnet/data/processed/target_data/temp_target_kinetic_data.csv


In [45]:
target_data_pd = pd.read_csv(out_csv)
target_data_pd[target_data_pd['label'] == 'k_for (TST+T)'].describe()

,A,n,Ea,dA,dn,dEa,A_log10,Ea_yj
count,1.848000e+03,1848.000000,1848.000000,1848.000000,1848.000000,1848.000000,1848.000000,1.848000e+03
mean,1.222523e+09,4.328233,29.157218,1.902049,0.069837,0.399380,-2.736038,3.075943e-17
std,5.251302e+10,0.807474,31.297356,2.267665,0.050362,0.288009,3.319707,1.000271e+00
min,4.856850e-30,1.563490,-7.952310,1.013580,0.001757,0.010050,-29.313645,-3.286445e+00
25%,8.707215e-06,3.892190,6.807783,1.234862,0.027478,0.157137,-5.060128,-6.180919e-01
50%,8.341260e-04,4.407760,18.278100,1.640895,0.064506,0.368890,-3.078768,-1.096539e-01
75%,9.697212e-02,4.832607,43.171575,2.157915,0.100182,0.572911,-1.013356,6.463282e-01
max,2.257450e+12,11.588700,210.310000,85.778800,0.579848,3.315990,12.353618,3.498660e+00


In [46]:
feat_data = Featuriser(os.path.expanduser("~/code/chemprop_phd_customised/habnet/data/processed/sdf_data"), filter_rules={'label': 'k_for (TST+T)'},
                       path = out_csv, set_col_index=False, target_col=['A_log10','n', 'Ea_yj'],
                       include_extra_features = False)

Reaction ID rmg_rxn_15513_fixed not found in target data. Skipping this SDF file.
Reaction ID rxn_1480_fixed not found in target data. Skipping this SDF file.
Reaction ID rxn_721_fixed not found in target data. Skipping this SDF file.
Reaction ID rxn_1506_fixed not found in target data. Skipping this SDF file.
Reaction ID rxn_754_fixed not found in target data. Skipping this SDF file.
Reaction ID rxn_275_fixed not found in target data. Skipping this SDF file.
Reaction ID rxn_1126_fixed not found in target data. Skipping this SDF file.
Reaction ID rmg_rxn_16514_fixed not found in target data. Skipping this SDF file.
Reaction ID rmg_rxn_16785 not found in target data. Skipping this SDF file.
Reaction ID rmg_rxn_156_fixed not found in target data. Skipping this SDF file.
Reaction ID rxn_982_fixed not found in target data. Skipping this SDF file.
Reaction ID rxn_483_fixed not found in target data. Skipping this SDF file.
Reaction ID rmg_rxn_16807_fixed not found in target data. Skipping th

In [47]:
len(feat_data[0])

1699

In [48]:
feat_data[0]

[MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x738ff50835a0>, y=[1.5442938187845985, 3.29395, -2.133882692706495], weight=1.0, gt_mask=None, lt_mask=None, x_d=None, x_phase=None, name='rmg_rxn_86_r1h', V_f=None, E_f=None, V_d=None),
 MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x738ff50837d0>, y=[5.627936005825284, 2.46562, -0.2997509089798689], weight=1.0, gt_mask=None, lt_mask=None, x_d=None, x_phase=None, name='rxn_191_r1h', V_f=None, E_f=None, V_d=None),
 MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x738ff5083d80>, y=[-3.534758465978714, 4.49305, 0.4625016001739204], weight=1.0, gt_mask=None, lt_mask=None, x_d=None, x_phase=None, name='rxn_618_r1h', V_f=None, E_f=None, V_d=None),
 MoleculeDatapoint(mol=<rdkit.Chem.rdchem.Mol object at 0x738ff5083c30>, y=[-1.185020981213695, 4.14783, -0.7217817036654244], weight=1.0, gt_mask=None, lt_mask=None, x_d=None, x_phase=None, name='rxn_1833_r1h', V_f=None, E_f=None, V_d=None),
 MoleculeDatapoint(mol=<rdkit.C

In [49]:
for i in range(len(feat_data)):
    if len(feat_data[0][i].y) != 3:
        print(f"Index {i} has y of length {len(feat_data[i].y)}: {feat_data[i].y}")


In [50]:
component_to_split_by = 0
mols = [d.mol for d in feat_data[component_to_split_by]]

train_indices, val_indices, test_indices = data.make_split_indices(mols, "kennard_stone", (0.8, 0.1, 0.1))


train_data, val_data, test_data = data.split_data_by_indices(
    feat_data, train_indices, val_indices, test_indices
)

The return type of make_split_indices has changed in v2.1 - see help(make_split_indices)


In [51]:
rxn_ids = []
for i in range(len(train_data[0][0])):
    rxn_name = train_data[0][0][i].name
    # Need to remove _r1h or _r2h from the reaction name
    rxn_name = rxn_name.replace('_r1h', '').replace('_r2h', '')
    rxn_ids.append(rxn_name)

In [52]:
print("Molecule R1H: ", feat_data[0][0].name)
print("Molecule R2H: ", feat_data[1][0].name)

Molecule R1H:  rmg_rxn_86_r1h
Molecule R2H:  rmg_rxn_86_r2h


In [53]:
from rdkit.Chem.rdchem import HybridizationType
from rdkit.Chem import SanitizeMol
# for j in range(len(feat_data[0])):
#     mol = feat_data[0][j].mol
#     for i in range(mol.GetNumAtoms()):
#         atom = mol.GetAtomWithIdx(i)
#         hyb = atom.GetHybridization()
#         if hyb == HybridizationType.UNSPECIFIED:
#             print(f"Atom {i} has unspecified hybridization.")

for j in range(len(feat_data[1])):
    mol = feat_data[1][j].mol

    for i in range(mol.GetNumAtoms()):
        atom = mol.GetAtomWithIdx(i)
        hyb = atom.GetHybridization()
        if hyb == HybridizationType.UNSPECIFIED:
            print(f"Atom {i} has unspecified hybridization.")
            print(feat_data[1][j].name)




In [54]:
rxn_ids = []
for i in range(len(train_data[0][0])):
    rxn_name = train_data[0][0][i].name
    # Need to remove _r1h or _r2h from the reaction name
    rxn_name = rxn_name.replace('_r1h', '').replace('_r2h', '')
    rxn_ids.append(rxn_name)

In [55]:
# Read extra info
import pandas as pd
atom_extra_feats = pd.read_csv("/home/calvin/code/chemprop_phd_customised/habnet/data/preprocessing/all_sdf_features.csv")
atom_extra_feats.columns

Index(['rxn_id', 'mol_type', 'focus_atom_idx', 'path', 'radius', 'angle',
       'dihedral', 'r_exist', 'a_exist', 'd_exist', 'focus_atom_symbol',
       'is_donor', 'is_acceptor', 'is_donor_H', 'is_acceptor_neighbor'],
      dtype='object')

In [56]:
import numpy as np

def rbf_expand(values, num_centers=20, r_min=None, r_max=None, gamma=None):
    values = np.asarray(values)
    if r_min is None:
        r_min = float(np.min(values))
    if r_max is None:
        r_max = float(np.max(values))
    # Generate evenly spaced centers
    centers = np.linspace(r_min, r_max, num_centers)
    if gamma is None:
        # Set gamma so adjacent bases overlap well
        gamma = 1.0 / (centers[1] - centers[0])**2
    # Compute RBF
    expanded = np.exp(-gamma * (values[..., None] - centers)**2)
    return expanded  # shape: (len(values), num_centers)

def dihedral_to_sin_cos(dihedrals_deg):
    dihedrals_rad = np.deg2rad(dihedrals_deg)
    sin_vals = np.sin(dihedrals_rad)
    cos_vals = np.cos(dihedrals_rad)
    return np.stack([sin_vals, cos_vals], axis=-1)

def normalize_angle(angle_deg, a_min=0.0, a_max=180.0):
    # If your angles can go up to 180, otherwise adjust a_max as needed
    angle_deg = np.asarray(angle_deg)
    return (angle_deg - a_min) / (a_max - a_min)



In [57]:
import numpy as np
import pandas as pd

# Your DataFrame and rxn_ids
# atom_extra_feats: DataFrame with all data
# rxn_ids: list of rxn_ids in the train set

# 1. Subset for fitting
train_mask = atom_extra_feats['rxn_id'].isin(rxn_ids)
train_feats = atom_extra_feats[train_mask]

# 2. Compute parameters from training data
num_centers = 16
r_min = train_feats['radius'].min()
r_max = train_feats['radius'].max()
a_min = train_feats['angle'].min()
a_max = train_feats['angle'].max()

# 3. Apply to all data using train params
radius_rbf = rbf_expand(atom_extra_feats['radius'].values, num_centers=num_centers, r_min=r_min, r_max=r_max)
dihedral_sc = dihedral_to_sin_cos(atom_extra_feats['dihedral'].values)
angle_norm = normalize_angle(atom_extra_feats['angle'].values, a_min=a_min, a_max=a_max)

# 4. Stack or add to DataFrame
for i in range(num_centers):
    atom_extra_feats[f'radius_rbf_{i}'] = radius_rbf[:, i]
atom_extra_feats['dihedral_sin'] = dihedral_sc[:, 0]
atom_extra_feats['dihedral_cos'] = dihedral_sc[:, 1]
atom_extra_feats['angle_norm'] = angle_norm


In [58]:
def get_atom_feats_for_dp(dp_name, atom_extra_feats):
    rxn_id, mol_type = dp_name.rsplit('_', 1)
    subset = atom_extra_feats[
        (atom_extra_feats['rxn_id'] == rxn_id) &
        (atom_extra_feats['mol_type'] == mol_type)
    ]
    # Sort by focus_atom_idx
    subset = subset.sort_values("focus_atom_idx")
    return subset


def attach_morgan_to_dps(dps):
    new_dps = []
    drop_cols = ['rxn_id', 'mol_type', 'focus_atom_idx', 'path', 'radius', 'angle', 'dihedral','focus_atom_symbol']
    for dp in dps:
        # pick the mol you actually want to fingerprint:
        mol = dp.mol if not isinstance(dp.mol, tuple) else dp.mol[0]
        extra_atom_feats = get_atom_feats_for_dp(dp.name, atom_extra_feats).drop(columns=drop_cols).values
        # Make sure extra_atom_feats is not empty
        if extra_atom_feats.shape[0] == 0:
            print(f"No extra atom features found for datapoint {dp.name}. Check your atom_extra_feats DataFrame.")
            continue
        new_dp = data.MoleculeDatapoint(
            mol=dp.mol,
            y=dp.y,
            weight=dp.weight,
            gt_mask=dp.gt_mask,
            lt_mask=dp.lt_mask,
            V_f=extra_atom_feats,
            E_f=dp.E_f,
            V_d=dp.V_d,
            x_d=dp.x_d,      # <-- lowercase x_d
            x_phase=dp.x_phase,
            name=dp.name
        )
        new_dps.append(new_dp)
    return new_dps


featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer(extra_atom_fdim=26)

train_datasets = [
    data.MoleculeDataset(
        attach_morgan_to_dps(train_data[0][i]),
        featurizer
    )
    for i in range(len(MOL_TYPES))
]

val_datasets = [
    data.MoleculeDataset(
        attach_morgan_to_dps(val_data[0][i]),
        featurizer
    )
    for i in range(len(MOL_TYPES))
]
test_datasets = [
    data.MoleculeDataset(
        attach_morgan_to_dps(test_data[0][i]),
        featurizer
    )
    for i in range(len(MOL_TYPES))
]


No extra atom features found for datapoint rmg_rxn_16786_r1h. Check your atom_extra_feats DataFrame.
No extra atom features found for datapoint rmg_rxn_1349_r1h. Check your atom_extra_feats DataFrame.
No extra atom features found for datapoint rmg_rxn_16786_r2h. Check your atom_extra_feats DataFrame.
No extra atom features found for datapoint rmg_rxn_1349_r2h. Check your atom_extra_feats DataFrame.
No extra atom features found for datapoint rxn_2010_r1h. Check your atom_extra_feats DataFrame.
No extra atom features found for datapoint rmg_rxn_16776_r1h. Check your atom_extra_feats DataFrame.
No extra atom features found for datapoint rxn_2010_r2h. Check your atom_extra_feats DataFrame.
No extra atom features found for datapoint rmg_rxn_16776_r2h. Check your atom_extra_feats DataFrame.


In [59]:
train_datasets[1].data[0].V_f.shape

(14, 26)

In [60]:
# file: my_losses.py
import torch
import torch.nn.functional as F
from chemprop.nn.metrics import ChempropMetric, LossFunctionRegistry

@LossFunctionRegistry.register("huber_mse")
class HuberMSE(ChempropMetric):
    """
    A mixed loss that applies:
      – Huber (smooth-L1) on the first task (log10(A))
      – MSE on the remaining tasks (n, Ea)
    and then weights each task via task_weights.
    """
    def __init__(self,
                 task_weights: list[float] = 1.0,
                 delta: float = 1.0):
        super().__init__(task_weights)
        self.delta = delta
        self.register_buffer("delta_buf", torch.tensor(delta))

    def _calc_unreduced_loss(self, preds: torch.Tensor, targets: torch.Tensor, *args):
        # preds, targets: shape (B, 3)
        # 1) Huber on the 0th channel
        diff0    = preds[:, 0] - targets[:, 0]
        absdiff0 = diff0.abs()
        mask0    = absdiff0 <= self.delta_buf
        huber0   = torch.where(mask0,
                               0.5 * diff0.pow(2),
                               self.delta_buf * (absdiff0 - 0.5 * self.delta_buf))

        # 2) MSE on channels 1 and 2
        mse12    = (preds[:, 1:] - targets[:, 1:]).pow(2)

        # 3) concatenate back into (B,3)
        return torch.cat([huber0.unsqueeze(1), mse12], dim=1)


In [61]:
train_mcdset = data.MulticomponentDataset(train_datasets)
scaler = train_mcdset.normalize_targets()

val_mcdset = data.MulticomponentDataset(val_datasets)
val_mcdset.normalize_targets(scaler)
test_mcdset = data.MulticomponentDataset(test_datasets)
test_mcdset.normalize_targets(scaler)

StandardScaler()

In [62]:
print("Number of node feats.: ", train_datasets[0][0].mg.V.shape[1])
print("Number of edge feats.: ", train_datasets[0][0].mg.E.shape[1])

Number of node feats.:  98
Number of edge feats.:  14


In [63]:
train_loader = data.build_dataloader(train_mcdset, batch_size=64, shuffle=True, num_workers=8, pin_memory=True)
val_loader = data.build_dataloader(val_mcdset, shuffle=False, batch_size=64)
test_loader = data.build_dataloader(test_mcdset, shuffle=False, batch_size=64)
output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)


In [64]:
# mcmp = nn.MulticomponentMessagePassing(
#     blocks=[nn.BondMessagePassing(depth=4, dropout=0.2, d_v=featurizer.atom_fdim, d_e=featurizer.bond_fdim, d_h=1024,
#                                   ) for _ in range(len(MOL_TYPES))],
#     n_components=len(MOL_TYPES), shared = False    
# )

In [65]:
agg = nn.MeanAggregation()

In [66]:
mcmp = nn.MulticomponentMessagePassing(
    blocks=[nn.BondMessagePassing(depth=4, dropout=0.2, d_v=featurizer.atom_fdim, d_e=featurizer.bond_fdim, d_h=512,
                                  ) for _ in range(len(MOL_TYPES))],
    n_components=len(MOL_TYPES), shared = False    
)

agg = nn.MeanAggregation()
ffn = nn.RegressionFFN(
    n_tasks=3,
    input_dim=mcmp.output_dim,
    # output_transform=output_transform,
    dropout=0.2,
    criterion=HuberMSE(task_weights=[10.0, 1.0, 1.0], delta=0.1),  # Use custom Huber loss
)
metric_list = [HuberMSE(task_weights=[10.0, 1.0, 1.0], delta=0.1),metrics.RMSE(task_weights=[10.0, 1.0, 1.0]), metrics.MAE([10.0, 1.0, 1.0]), metrics.R2Score([10.0, 1.0, 1.0])] # Only the first metric is used for training and early stopping
mcmpnn = multi.MulticomponentMPNN(
    mcmp,
    agg,
    ffn,
    metrics=metric_list,
)

mcmpnn



MulticomponentMPNN(
  (message_passing): MulticomponentMessagePassing(
    (blocks): ModuleList(
      (0-1): 2 x BondMessagePassing(
        (W_i): Linear(in_features=112, out_features=512, bias=False)
        (W_h): Linear(in_features=512, out_features=512, bias=False)
        (W_o): Linear(in_features=610, out_features=512, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
        (tau): ReLU()
        (V_d_transform): Identity()
        (graph_transform): Identity()
      )
    )
  )
  (agg): MeanAggregation()
  (bn): Identity()
  (predictor): RegressionFFN(
    (ffn): MLP(
      (0): Sequential(
        (0): Linear(in_features=1024, out_features=300, bias=True)
      )
      (1): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.2, inplace=False)
        (2): Linear(in_features=300, out_features=3, bias=True)
      )
    )
    (criterion): HuberMSE(task_weights=[[10.0, 1.0, 1.0]])
    (output_transform): Identity()
  )
  (X_d_transform): Identity()
  (metrics): Mo

In [67]:


# Trainer setup
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
checkpoint_callback = ModelCheckpoint(
    monitor="val/rmse",
    mode="min",
    save_top_k=1,
    filename="best_model",
)
early_stopping_callback = EarlyStopping(
    monitor="val/rmse",
    patience=10,  # Number of epochs with no improvement after which training will be stopped
    mode="min",
)


# CSV logger setup
from lightning.pytorch.loggers import CSVLogger
csv_logger = CSVLogger(
    save_dir="logs",
    name="mcmpnn_experiment",
    version="v1"
)

trainer = pl.Trainer(
    logger=csv_logger,
    enable_checkpointing=True,
    enable_progress_bar=True,
    accelerator="auto",
    callbacks=[checkpoint_callback, early_stopping_callback],
    devices=1,
    max_epochs=200, # number of epochs to train for
)



GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [68]:
batch = next(iter(train_loader))
preds = mcmpnn(batch[0], batch[1], batch[2]).detach().cpu().numpy()
trues = batch[3].cpu().numpy()
print("preds[:5]:", preds[:5])
print("trues[:5]:", trues[:5])

preds[:5]: [[ 0.00444836 -0.00985329  0.08133124]
 [ 0.03875706  0.04145657  0.07931454]
 [ 0.03044819  0.00402354  0.08939967]
 [ 0.02963689  0.01200867  0.09632017]
 [-0.00635336 -0.0282146   0.06937996]]
trues[:5]: [[ 0.1271692   0.28415817  1.554719  ]
 [-1.1588774   1.0786508   0.80863535]
 [ 0.4758436  -0.93045366 -1.2014465 ]
 [ 2.6659627  -2.591772   -0.630505  ]
 [ 0.72565943 -0.4464993  -0.97296953]]


In [69]:
trainer.fit(mcmpnn, train_loader, val_loader)

/home/calvin/miniforge3/envs/habnet_rocm/lib/python3.12/site-packages/lightning/fabric/loggers/csv_logs.py:268: Experiment logs directory logs/mcmpnn_experiment/v1 exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
/home/calvin/miniforge3/envs/habnet_rocm/lib/python3.12/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory logs/mcmpnn_experiment/v1/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.


/home/calvin/miniforge3/envs/habnet_rocm/lib/python3.12/site-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (22) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.

  | Name            | Type                         | Params | Mode 
-------------------------------------------------------------------------
0 | message_passing | MulticomponentMessagePassing | 1.3 M  | train
1 | agg             | MeanAggregation              | 0      | train
2 | bn              | Identity                     | 0      | train
3 | predictor       | RegressionFFN                | 308 K  | train
4 | X_d_transform   | Identity                     | 0      | train
5 | metrics         | ModuleList                   | 0      | train
-------------------------------------------------------------------------
1.6 M     Trainable params
0         Non-trainable params
1.6 M     Tota

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/calvin/miniforge3/envs/habnet_rocm/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [70]:
batch = next(iter(train_loader))
preds = mcmpnn(batch[0], batch[1], batch[2]).detach().cpu().numpy()
trues = batch[3].cpu().numpy()
print("preds[:5]:", preds[:5])
print("trues[:5]:", trues[:5])

preds[:5]: [[-0.42464375  0.7217953  -0.3713733 ]
 [ 0.33403146 -0.3169172  -0.94729304]
 [-0.5644946   0.4149239  -0.42155728]
 [-0.4693913   0.4343918   0.0108362 ]
 [-0.00861194 -0.03030222  1.7217946 ]]
trues[:5]: [[-0.41257742  0.7697663  -0.26887512]
 [ 0.39692658 -0.24524918 -0.6559584 ]
 [-0.5789109   0.42845526 -0.26452717]
 [-0.52219677  0.47149125 -0.04672731]
 [-0.0976873   0.13846175  1.9082043 ]]


In [71]:
trainer.test(mcmpnn, test_loader) 

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/calvin/miniforge3/envs/habnet_rocm/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test/huber_mse       │     0.259957879781723     │
│         test/mae          │    1.0612962245941162     │
│          test/r2          │    0.7118161916732788     │
│         test/rmse         │    1.0153449773788452     │
└───────────────────────────┴───────────────────────────┘

[{'test/huber_mse': 0.259957879781723,
  'test/rmse': 1.0153449773788452,
  'test/mae': 1.0612962245941162,
  'test/r2': 0.7118161916732788}]

In [72]:
# Grab one batch:
batch = next(iter(train_loader))
ys = batch.Y        # shape: (batch_size, 3)
print("y shape:", ys.shape)
print("y min / max per column:", ys.min(0).values, ys.max(0).values)

y shape: torch.Size([64, 3])
y min / max per column: tensor([-1.2356, -3.1436, -2.5695]) tensor([3.2996, 1.4907, 3.0206])


In [73]:
import torch
import numpy as np
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score

# 0. ensure pt_ea is your fitted PowerTransformer for Ea
#    and trainer, mcmpnn, test_loader are in scope

# 1. Predict in scaled (z-score) space
mcmpnn.eval()
with torch.no_grad():
    scaled_preds_batches = trainer.predict(mcmpnn, dataloaders=test_loader)
scaled_preds = torch.cat(scaled_preds_batches, dim=0).cpu().numpy()   # (N,3)

# 2. Un-scale back to [log10(A), n, Ea_yj]
scaler   = output_transform.to_standard_scaler()
unscaled = scaler.inverse_transform(scaled_preds)                     # (N,3)

# 3. Grab the true y’s and un-scale them
y_scaled = []
for batch in test_loader:
    y_scaled.append(batch.Y)
y_scaled       = torch.cat(y_scaled, dim=0).cpu().numpy()
unscaled_true  = scaler.inverse_transform(y_scaled)                   # (N,3)

# 4. Recover raw A and raw Ea
logA_pred   = unscaled[:,   0]
n_pred      = unscaled[:,   1]
Ea_yj_pred  = unscaled[:,   2]

logA_true   = unscaled_true[:,   0]
n_true      = unscaled_true[:,   1]
Ea_yj_true  = unscaled_true[:,   2]

# exponentiate the A column
A_pred = 10 ** logA_pred
A_true = 10 ** logA_true

# invert the Yeo–Johnson on Ea
Ea_pred = pt_ea.inverse_transform(Ea_yj_pred.reshape(-1,1)).ravel()
Ea_true = pt_ea.inverse_transform(Ea_yj_true.reshape(-1,1)).ravel()

# 5. Clamp to physically‐reasonable bounds
epsilon = 1e-8
A_pred  = np.clip(A_pred, epsilon, None)
Ea_pred = np.clip(Ea_pred, 0.0,    None)

# 6. Stack into final arrays: [A, n, Ea]
preds_final = np.vstack([A_pred,    n_pred,    Ea_pred]).T
trues_final = np.vstack([A_true,    n_true,    Ea_true]).T

# 7. Compute raw‐unit metrics
mae_vals   = mean_absolute_error   (trues_final, preds_final, multioutput='raw_values')
rmse_vals  = root_mean_squared_error(trues_final, preds_final, multioutput='raw_values')
r2_vals    = r2_score              (trues_final, preds_final, multioutput='raw_values')

# 8. Compute log-space R² for A
r2_logA    = r2_score(logA_true, logA_pred)

# 9. Compute % error for A
mae_A      = mae_vals[0]
mean_A     = np.mean(A_true)
pct_err_A  = 100 * mae_A / mean_A

# 10. Print everything
print("=== Raw-unit metrics ===")
print(" MAE [A,   n,    Ea]:", mae_vals)
print(" RMSE[A,   n,    Ea]:", rmse_vals)
print("  R2 [A,   n,    Ea]:", r2_vals)

print("\n=== A-specific diagnostics ===")
print(f" R2 on log10(A):        {r2_logA:.3f}")
print(f" MAE(A) = {mae_A:.2e},  which is {pct_err_A:.1f}% of mean A")
print(f" A_pred   range: {A_pred.min():.2e} … {A_pred.max():.2e}")
print(f" n_pred   range: {n_pred.min():.2f} … {n_pred.max():.2f}")
print(f" Ea_pred  range: {Ea_pred.min():.2f} … {Ea_pred.max():.2f}")


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/calvin/miniforge3/envs/habnet_rocm/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

=== Raw-unit metrics ===
 MAE [A,   n,    Ea]: [4.8778232e+03 2.6664761e-01 5.8411183e+00]
 RMSE[A,   n,    Ea]: [3.5852215e+04 5.0512856e-01 9.6613264e+00]
  R2 [A,   n,    Ea]: [0.7831224  0.55175894 0.9114224 ]

=== A-specific diagnostics ===
 R2 on log10(A):        0.702
 MAE(A) = 4.88e+03,  which is 59.9% of mean A
 A_pred   range: 1.30e-08 … 5.83e+05
 n_pred   range: 2.40 … 5.83
 Ea_pred  range: 0.00 … 110.70


In [74]:
from sklearn.metrics import mean_absolute_error

mae_logA = mean_absolute_error(logA_true, logA_pred)
print(f"MAE on log10(A) = {mae_logA:.3f} dex  → ~{10**mae_logA:.2f}× fold‐error")


MAE on log10(A) = 0.844 dex  → ~6.98× fold‐error


In [75]:
import numpy as np

mape = np.median(np.abs((A_pred - A_true) / A_true)) * 100
print(f"Median absolute % error on A = {mape:.1f}%")


Median absolute % error on A = 80.9%


In [76]:
mask = A_true > 1e-6
mape = np.mean(np.abs((A_pred[mask] - A_true[mask]) / A_true[mask])) * 100
print(f"Filtered MAPE (A>1e-6) = {mape:.1f}%")


Filtered MAPE (A>1e-6) = 2734.0%
